# Currency Ising Model Optimisation

### Import Libraries

In [20]:
# Import necessary libraries for data manipulation, numerical processing, and optimisation
import numpy as np
import pandas as pd
from scipy.optimize import minimize

### Load Data and Prepare Matrix

In [21]:
# Load the currency data from an csv file into a pandas DataFrame
df = pd.read_csv("Results/data_matrix.csv")

# Extract column names from the DataFrame excluding any non-currency columns like 'Date'
symbols = df.columns.tolist()[1:]

# Remove the 'Date' column to focus solely on currency data and convert to a numpy matrix for analysis
data_matrix = df.drop(columns=["Date"]).to_numpy()

# Output the shape of the matrix to ensure it has the expected dimensions
print(f"The shape of the data matrix is: {data_matrix.shape}")

The shape of the data matrix is: (4538, 26)


## 1. Subset Optimisation (6/7/6/7):
- Divide the full J matrix and h vector into 4 subsets according to the proposed configuration.
- Optimise each subset multiple times to construct a symmetric J matrix and an h vector for all 26 currencies.
- Display the initial guesses for J and h with appropriate significant figures for clarity. 


### Define and Extract Currency Subsets

In [22]:
# Define indices for currency subsets to be used in separate optimisation analyses
subsets_indices = {
    'subset_1': list(range(6)),     # First subset (first 6 currencies)
    'subset_2': list(range(6, 13)), # Second subset (next 7 currencies)
    'subset_3': list(range(13, 19)), # Third subset (next 6 currencies)
    'subset_4': list(range(19, 26))  # Fourth subset (last 7 currencies)
}

# Extract the data subsets for analysis based on the defined indices
data_subsets = {key: data_matrix[:, indices] for key, indices in subsets_indices.items()}

### Define Log-Pseudo-Likelihood Function

In [23]:
# Define a function to calculate the log-pseudo-likelihood and its gradients for optimisation

def log_pseudolikelihood_and_gradients(J, h, X):
    """
    Calculate the log-pseudolikelihood for the Ising model and its gradients.
    
    Args:
    J: Interaction matrix (J matrix).
    h: External field vector (h vector).
    X: Spin configurations (data matrix).
    
    Returns:
    Tuple of negative log-pseudolikelihood, gradients of J, and gradients of h.
    """
    n, d = X.shape  # Number of samples and dimensions
    log_likelihood = 0
    grad_J = np.zeros_like(J)  # Initialise the gradient of J
    grad_h = np.zeros_like(h)  # Initialise the gradient of h

    for i in range(n):
        for j in range(d):
            S_ij = np.dot(J[j, :], X[i, :]) - J[j, j] * X[i, j] + h[j]
            log_likelihood += X[i, j] * S_ij - np.log(2 * np.cosh(S_ij))
            grad_h[j] += X[i, j] - np.tanh(S_ij)
            for k in range(d):
                if k != j:
                    grad_J[j, k] += X[i, j] * X[i, k] - np.tanh(S_ij) * X[i, k]
    
    grad_J = (grad_J + grad_J.T) / 2  # Make the J gradient symmetric

    # Return the negative likelihood and gradients for minimisation
    return -log_likelihood, -grad_J, -grad_h

### Subset Optimisation Function

In [24]:
# Define the optimisation function using the L-BFGS-B algorithm

def optimise_subset(data_subset, J_subset, h_subset):
    """
    Optimise a subset of the Ising model using the L-BFGS-B algorithm.
    
    Args:
    data_subset: Subset of data for optimisation.
    J_subset: Initial guess for the interaction matrix (J matrix) for the subset.
    h_subset: Initial guess for the external field vector (h vector) for the subset.
    
    Returns:
    Optimised interaction matrix (J matrix), external field vector (h vector), and a success flag.
    """
    N = J_subset.shape[0]  # Number of spins (currencies)

    # Flatten the J matrix and h vector for the optimisation
    x0 = np.concatenate([J_subset[np.triu_indices(N, k=1)], h_subset])

    def objective_function(x):
        # Construct the symmetric J matrix and calculate the likelihood and gradients
        J, h = reconstruct_J_and_h(x, N)
        likelihood, grad_J, grad_h = log_pseudolikelihood_and_gradients(J, h, data_subset)
        # Combine and return the likelihood and flattened gradients
        return likelihood, np.concatenate([grad_J[np.triu_indices(N, k=1)], grad_h])
    
    # Execute the optimisation using the objective function and initial guesses
    res = minimize(objective_function, x0, method='L-BFGS-B', jac=True)

    # Reconstruct the optimised J matrix and h vector from the optimisation result
    J_optimised, h_optimised = reconstruct_J_and_h(res.x, N)

    return J_optimised, h_optimised, res.success

# Helper function to reconstruct the symmetric J matrix and h vector from a flattened array
def reconstruct_J_and_h(flattened_array, N):
    # Extract the upper triangular part of J and the h vector from the flattened array
    J_upper_tri = flattened_array[:N * (N - 1) // 2]
    h = flattened_array[N * (N - 1) // 2:]

    # Construct the symmetric J matrix from the upper triangular part
    J = np.zeros((N, N))
    J[np.triu_indices(N, k=1)] = J_upper_tri
    J += J.T  # Symmetrise the J matrix
    np.fill_diagonal(J, 0)  # Set the diagonal to 0 as there are no self-interactions

    return J, h

###  Initialise and Optimise First Subset

In [25]:
# Initialise the J matrix and h vector for the first subset for optimisation
J_initial_subset_1 = np.zeros((6, 6))  # 6x6 matrix for the first 6 currencies
h_initial_subset_1 = np.zeros(6)       # Vector of length 6 for the external field

# Optimise the first subset using the initialised J matrix and h vector
J_subset_optimised_1, h_subset_optimised_1, optimisation_success_1 = optimise_subset(
    data_subsets['subset_1'], J_initial_subset_1, h_initial_subset_1)

# Display the optimised J matrix, h vector, and whether the optimisation was successful
print("Optimised J matrix for subset 1:\n", J_subset_optimised_1)
print("Optimised h vector for subset 1:\n", h_subset_optimised_1)
print("Optimisation successful for subset 1:\n", optimisation_success_1)

Optimised J matrix for subset 1:
 [[ 0.          0.15363498  0.1863861   0.01296148  0.53258729  0.30021676]
 [ 0.15363498  0.          0.34385924  0.17001976  0.15934126  0.11217307]
 [ 0.1863861   0.34385924  0.          0.08919781  0.0852463   0.15266204]
 [ 0.01296148  0.17001976  0.08919781  0.          0.07359777 -0.07208994]
 [ 0.53258729  0.15934126  0.0852463   0.07359777  0.          0.17436713]
 [ 0.30021676  0.11217307  0.15266204 -0.07208994  0.17436713  0.        ]]
Optimised h vector for subset 1:
 [ 0.04166438  0.00674824 -0.02915886 -0.03756775  0.01753722 -0.01687403]
Optimisation successful for subset 1:
 True


### Iterative Optimisation of Currency Subsets (6/7/6/7)

In [26]:
# Initialise storage for optimised J matrices, h vectors, and success indicators for each subset
optimised_J_subsets = {}
optimised_h_subsets = {}
optimisation_success_subsets = {}

# Iterate over each subset and carry out the optimisation process
for subset_name, data_subset in data_subsets.items():
    # Find out the number of currencies in the current subset
    subset_size = len(subsets_indices[subset_name])
    
    # Set up initial guesses for the J matrix and h vector for the optimisation
    J_initial = np.zeros((subset_size, subset_size))  # Initialise a square matrix of zeros for J
    h_initial = np.zeros(subset_size)                 # Initialise a zero vector for h
    
    # Conduct the optimisation using the initial guesses and the current data subset
    J_optimised, h_optimised, optimisation_success = optimise_subset(
        data_subset,
        J_initial,
        h_initial
    )
    
    # If the optimisation is successful, store the resulting J and h in their respective dictionaries
    if optimisation_success:
        optimised_J_subsets[subset_name] = J_optimised
        optimised_h_subsets[subset_name] = h_optimised
    # Record the success status of the optimisation for each subset
    optimisation_success_subsets[subset_name] = optimisation_success

# Display the optimised J matrices, h vectors, and success status for each subset
print("Optimised J matrices for each subset:\n", optimised_J_subsets)
print("Optimised h vectors for each subset:\n", optimised_h_subsets)
print("Optimisation success status for each subset:\n", optimisation_success_subsets)

Optimised J matrices for each subset:
 {'subset_1': array([[ 0.        ,  0.15363498,  0.1863861 ,  0.01296148,  0.53258729,
         0.30021676],
       [ 0.15363498,  0.        ,  0.34385924,  0.17001976,  0.15934126,
         0.11217307],
       [ 0.1863861 ,  0.34385924,  0.        ,  0.08919781,  0.0852463 ,
         0.15266204],
       [ 0.01296148,  0.17001976,  0.08919781,  0.        ,  0.07359777,
        -0.07208994],
       [ 0.53258729,  0.15934126,  0.0852463 ,  0.07359777,  0.        ,
         0.17436713],
       [ 0.30021676,  0.11217307,  0.15266204, -0.07208994,  0.17436713,
         0.        ]]), 'subset_2': array([[0.        , 0.03150901, 0.12884488, 0.49855793, 0.03136831,
        0.10086422, 0.04214823],
       [0.03150901, 0.        , 0.03867503, 0.05117874, 0.06978797,
        0.03110047, 0.11163803],
       [0.12884488, 0.03867503, 0.        , 0.61624577, 0.00190711,
        0.38373779, 0.0135196 ],
       [0.49855793, 0.05117874, 0.61624577, 0.        , 0.011

## 2. Larger Subset Optimisation (13/13):
- Use the updated  J  matrix and  h  vector from the converged subsets as initial guesses to optimise two larger subsets, each consisting of 13 currencies.
- Optimise these larger subsets until convergence. Each subset optimisation updates the J matrix and h vector.


### Matrix and Vector Aggregation for Optimisation

In [27]:
# Define functions to combine smaller matrices and vectors into larger, unified structures for optimisation analysis.

def combine_J_matrices(*matrices):
    """
    Combine smaller J matrices into a larger J matrix.
    Args:
    *matrices: An unpacked tuple of smaller J matrices.
    
    Returns:
    A combined larger J matrix.
    """
    # Calculate the total size for the new combined matrix
    size = sum(m.shape[0] for m in matrices)
    # Initialise the combined matrix with zeros
    J_combined = np.zeros((size, size))

    current_index = 0  # Starting index for combining matrices
    for m in matrices:
        m_size = m.shape[0]  # Size of the current matrix
        # Insert the current matrix into the combined matrix at the correct position
        J_combined[current_index:current_index+m_size, current_index:current_index+m_size] = m
        current_index += m_size  # Update the index for the next matrix

    return J_combined

def combine_h_vectors(*vectors):
    """
    Combine smaller h vectors into a larger h vector.
    Args:
    *vectors: An unpacked tuple of smaller h vectors.
    
    Returns:
    A combined larger h vector.
    """
    # Concatenate all vectors into a single, longer vector
    return np.concatenate(vectors)

### Combine Matrices and Vectors for Larger Subsets

In [28]:
# Combine matrices and vectors from previously optimised smaller subsets to form two larger 13x13 subsets for further optimisation.

# Combine the first two J matrices and h vectors to form the first larger subset
J_13x13_1 = combine_J_matrices(optimised_J_subsets['subset_1'], optimised_J_subsets['subset_2'])
h_13x13_1 = combine_h_vectors(optimised_h_subsets['subset_1'], optimised_h_subsets['subset_2'])

# Combine the last two J matrices and h vectors to form the second larger subset
J_13x13_2 = combine_J_matrices(optimised_J_subsets['subset_3'], optimised_J_subsets['subset_4'])
h_13x13_2 = combine_h_vectors(optimised_h_subsets['subset_3'], optimised_h_subsets['subset_4'])

# Print the combined matrices and vectors to verify their construction
print("Combined J matrix for the first 13x13 subset:\n", J_13x13_1)
print("Combined h vector for the first 13x13 subset:\n", h_13x13_1)
print("Combined J matrix for the second 13x13 subset:\n", J_13x13_2)
print("Combined h vector for the second 13x13 subset:\n", h_13x13_2)

Combined J matrix for the first 13x13 subset:
 [[ 0.          0.15363498  0.1863861   0.01296148  0.53258729  0.30021676
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.15363498  0.          0.34385924  0.17001976  0.15934126  0.11217307
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.1863861   0.34385924  0.          0.08919781  0.0852463   0.15266204
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.01296148  0.17001976  0.08919781  0.          0.07359777 -0.07208994
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.53258729  0.15934126  0.0852463   0.07359777  0.          0.17436713
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.30021676  0.11217307  0.15266204 -0.07208994  0.17436713  0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.  

### Optimisation of the First 13x13 Subset

In [29]:
# Perform optimisation on the first 13x13 subset using the combined J matrix and h vector as initial guesses.

# Extract the combined data of the first two subsets for optimisation
data_13x13_1 = np.hstack((data_subsets['subset_1'], data_subsets['subset_2']))

# Optimise the first 13x13 subset using a predefined function `optimise_subset`
J_13x13_optimised_1, h_13x13_optimised_1, optimisation_success_13x13_1 = optimise_subset(
    data_13x13_1,
    J_13x13_1,
    h_13x13_1
)

# Display the optimised J matrix, h vector, and success status for the first larger subset
print("Optimisation results for the first 13x13 subset:\n", J_13x13_optimised_1, h_13x13_optimised_1)
print("Optimisation successful:", optimisation_success_13x13_1)

Optimisation results for the first 13x13 subset:
 [[ 0.00000000e+00 -2.48091800e-02  1.57922812e-01  1.46600482e-02
   5.14589760e-01  2.71787486e-01 -1.39514014e-02  5.02724203e-02
   6.17500778e-02  6.96137276e-02  6.86732621e-02  1.49963080e-01
   1.13545507e-01]
 [-2.48091800e-02  0.00000000e+00  1.04496133e-01  9.10396175e-04
   1.18569886e-01 -5.85771413e-02  2.19469508e-01  4.37163033e-03
   3.51774337e-01  1.34394761e+00  8.16329758e-02  1.27915100e-01
   2.88826511e-02]
 [ 1.57922812e-01  1.04496133e-01  0.00000000e+00  6.55660896e-02
   7.14245787e-02  1.29924894e-01  9.90097063e-02  1.49091416e-02
   1.14654575e-01  8.86081884e-02  3.00446680e-02  9.03856006e-02
   1.70646328e-02]
 [ 1.46600482e-02  9.10396175e-04  6.55660896e-02  0.00000000e+00
   6.55205322e-02 -7.87198621e-02  2.92417911e-01  1.24078851e-02
   1.22733039e-02  5.76902975e-02  1.46680481e-02 -6.54215693e-02
  -1.01380459e-02]
 [ 5.14589760e-01  1.18569886e-01  7.14245787e-02  6.55205322e-02
   0.00000000e+0

### Optimisation of the Second 13x13 Subset

In [30]:
# Perform optimisation on the second 13x13 subset, following the same process as for the first subset.

# Combine the data for the third and fourth subsets for optimisation
data_13x13_2 = np.hstack((data_subsets['subset_3'], data_subsets['subset_4']))

# Optimise the second 13x13 subset using the same `optimise_subset` function
J_13x13_optimised_2, h_13x13_optimised_2, optimisation_success_13x13_2 = optimise_subset(
    data_13x13_2,
    J_13x13_2,
    h_13x13_2
)

# Display the optimised J matrix, h vector, and success status for the second larger subset
print("Optimisation results for the second 13x13 subset:\n", J_13x13_optimised_2, h_13x13_optimised_2)
print("Optimisation successful:", optimisation_success_13x13_2)

Optimisation results for the second 13x13 subset:
 [[ 0.          0.04882446  0.04840694 -0.01910868  0.04359987  0.08597943
   0.04810597  0.04952335  0.09290486  0.07760196  0.01296876  0.10823934
   0.04748053]
 [ 0.04882446  0.          0.10754786  0.02013554 -0.01322754  0.01537513
   0.13649823  0.11247997  0.01211739  0.08300673  0.07553298  0.05443731
   0.09986477]
 [ 0.04840694  0.10754786  0.          0.01441151  0.05749356  0.05901221
   0.13003002  0.06121611  0.01653348  0.19273527  0.17710194  0.04990353
   0.07435432]
 [-0.01910868  0.02013554  0.01441151  0.          0.11455467  0.05104383
   0.00086878  0.00950599  0.00314511  0.04759558  0.03825156 -0.00106092
   0.15913524]
 [ 0.04359987 -0.01322754  0.05749356  0.11455467  0.          0.02646624
   0.05021963  0.04649796  0.00224389  0.12054005  0.06061223  0.01639867
  -0.00538839]
 [ 0.08597943  0.01537513  0.05901221  0.05104383  0.02646624  0.
   0.02321516  0.11516793  0.53733089  0.15822078  0.08496776  0.009

## 3. Full Matrix Optimisation:
- Finally, use the converged  J  matrix and  h  vector from the 13/13 subsets as initial guesses for the optimisation of the full matrix.
- Run the full-matrix optimisation until convergence.


### Combine Optimised Subsets and Perform Full Optimisation

In [31]:
# Combine the optimised 13x13 J matrices into a single 26x26 J matrix for the full optimisation
J_initial_full = np.block([
    [J_13x13_optimised_1, np.zeros((13, 13))],  # Top-left block is the first optimised J matrix
    [np.zeros((13, 13)), J_13x13_optimised_2]   # Bottom-right block is the second optimised J matrix
])

# Combine the optimised h vectors into a single 26-dimensional h vector
h_initial_full = np.concatenate([h_13x13_optimised_1, h_13x13_optimised_2])

# Conduct the final optimisation for the entire set of currencies
J_optimised_full, h_optimised_full, success = optimise_subset(
    data_matrix,  # The full data matrix containing all currency pairs
    J_initial_full,
    h_initial_full
)

# Confirm the outcome of the full matrix optimisation
if success:
    print("Optimisation of the full matrix was successful.")
else:
    print("Optimisation of the full matrix did not converge.")

# Display the optimised full J matrix and h vector
print("Optimised full J matrix:\n", J_optimised_full)
print("Optimised full h vector:\n", h_optimised_full)

Optimisation of the full matrix was successful.
Optimised full J matrix:
 [[ 0.00000000e+00 -5.36594612e-02  1.14766267e-01  1.80898935e-02
   4.33767632e-01  2.04047524e-01 -4.96570304e-02  1.23368243e-02
   1.98088918e-02  2.72257756e-02  5.25456781e-02  7.70118928e-02
   6.24121512e-02 -9.31380938e-03 -3.66641756e-03  5.25847087e-02
   3.51880106e-03  2.88184428e-02  5.83618696e-02 -6.21224589e-03
   8.66078698e-02  8.62189971e-02  1.99489803e-01  2.54765794e-02
   4.45908182e-02  1.59310082e-01]
 [-5.36594612e-02  0.00000000e+00  9.53623706e-02  1.20807511e-02
   1.04051971e-01 -7.83026061e-02  1.97863460e-01 -7.07258263e-03
   3.37568921e-01  1.33642369e+00  7.86237097e-02  1.04659247e-01
   1.13893591e-02  3.04232999e-02 -5.81539622e-02  8.25985302e-02
  -2.34875978e-02  2.74707712e-02 -2.56296979e-02 -4.69738895e-02
   7.10323476e-02  1.24382355e-01 -5.12651718e-02  8.45791125e-02
   4.35568491e-02  1.98234452e-02]
 [ 1.14766267e-01  9.53623706e-02  0.00000000e+00  6.68083970e-0

### Convert to DataFrames and Save to CSV

In [32]:
# Convert the optimised J matrix and h vector into pandas DataFrames for easy manipulation and storage
J_df = pd.DataFrame(J_optimised_full, columns=symbols, index=symbols)
h_df = pd.DataFrame({'Symbol': symbols, 'h': h_optimised_full})

# Define the file paths for saving the csv files
J_matrix_file_path = 'Results/J_matrix.csv'
h_vector_file_path = 'Results/h_vector.csv'

# Save the J matrix and h vector to csv files for further analysis and record-keeping
J_df.to_csv(J_matrix_file_path, index=False)
h_df.to_csv(h_vector_file_path, index=False, header=False)

# Confirm that the files have been saved
print(f"The optimised J matrix has been saved to '{J_matrix_file_path}'.")
print(f"The optimised h vector has been saved to '{h_vector_file_path}'.")

The optimised J matrix has been saved to 'Results/J_matrix.csv'.
The optimised h vector has been saved to 'Results/h_vector.csv'.


In [33]:
print(f"Shape of data matrix is: {data_matrix.shape}")
print(f"Shape of optimised J matrix: {J_optimised_full.shape}")
print(f"Length of optimised h vector: {len(h_optimised_full)}")

Shape of data matrix is: (4538, 26)
Shape of optimised J matrix: (26, 26)
Length of optimised h vector: 26
